In [ ]:
#|default_exp toolloop

# Tool loop

In [ ]:
#| export
from claudette.core import *
from fastcore.utils import *

Anthropic provides an [interesting example](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/customer_service_agent.ipynb) of using tools to mock up a hypothetical ordering system. We're going to take it a step further, and show how we can dramatically simplify the process, whilst completing more complex tasks.

We'll start by defining the same mock customer/order data as in Anthropic's example, plus create a entity relationship between customers and orders:

In [ ]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
               orders=[orders['O3']])
}

We can now define the same functions from the original example -- but note that we don't need to manually create the large JSON schema, since Claudette handles all that for us automatically from the functions directly. We'll add some extra functionality to update order details when cancelling too.

In [ ]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

We're now ready to start our chat.

In [ ]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(models[-1], tools=tools)

We'll start with the same request as Anthropic showed:

In [ ]:
r = chat('Can you tell me the email address for customer C1?')
print(r.stop_reason)
r.content

tool_use


[ToolUseBlock(id='toolu_01VWYLUd1joTC9aF3HxBhRGG', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Claude asks us to use a tool. Claudette handles that automatically by just passing back the message:

In [ ]:
r = chat(r)
contents(r)

- Retrieving customer C1


'The email address for customer C1 is john@example.com.'

Let's consider a more complex case than in the original example -- what happens if a customer wants to cancel all of their orders?

In [ ]:
chat = Chat(models[-1], tools=tools)
r = chat('Please cancel all orders for customer C1 for me.')
print(r.stop_reason)
r.content

tool_use


[TextBlock(text="Okay, let's cancel all orders for customer C1:", type='text'),
 ToolUseBlock(id='toolu_01BCTBxaoR8RWGRuRaPmvjEQ', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

This is the start of a multi-stage tool use process. Doing it manually step by step is inconvenient, so let's write a function to handle this for us:

In [ ]:
#| exports
@patch
def toolloop(self:Chat,
             pr, # Prompt to pass to Claude
             max_steps=10, # Maximum number of tool requests to loop through
             temp=0, # Temperature
             maxtok=4096, # Maximum tokens
             stop:Optional[list[str]]=None, # Stop sequences
             trace_func:Optional[callable]=None, # Function to trace tool use steps (e.g `print`)
             cont_func:callable=None, # Function that stops loop if returns False
             **kw):
    "Add prompt `pr` to dialog and get a response from Claude, automatically following up with `tool_use` messages"
    r = self(pr, temp=temp, maxtok=maxtok, stop=stop, **kw)
    for i in range(max_steps):
        if r.stop_reason!='tool_use': break
        if trace_func: trace_func(r)
        r = self(r, temp=temp, maxtok=maxtok, stop=stop, **kw)
        if not cont_func(self.h[-2]['content']): i=max_steps
    if trace_func: trace_func(r)
    return r

We'll start by re-running our previous request - we shouldn't have to manually pass back the `tool_use` message any more:

In [ ]:
chat = Chat(models[-1], tools=tools)
r = chat.toolloop('Can you tell me the email address for customer C1?')
r

- Retrieving customer C1


The email address for customer C1 is john@example.com.

<details>

- id: msg_01LZnVXRnq6YVsYLxgi3jFbM
- content: [{'text': 'The email address for customer C1 is john@example.com.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 732, 'output_tokens': 19}

</details>

Let's see if it can handle the multi-stage process now -- we'll add `show_trace=print` to see each stage of the process:

In [ ]:
chat = Chat(models[-1], tools=tools)
r = chat.toolloop('Please cancel all orders for customer C1 for me.', trace_func=print)
r

ToolsBetaMessage(id='msg_01PgWFRC4eTFAFD7po126xvo', content=[TextBlock(text="Okay, let's cancel all orders for customer C1:", type='text'), ToolUseBlock(id='toolu_01U98A1o9PB3fSTRhJcej1wy', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 537; Out: 72; Total: 609)
- Retrieving customer C1
ToolsBetaMessage(id='msg_01RX3gCEsmiqMNUSaAboA9V7', content=[TextBlock(text="Based on the customer information, it looks like there are 2 orders for customer C1:\n- Order O1 for Widget A\n- Order O2 for Gadget B\n\nLet's cancel both of these orders:", type='text'), ToolUseBlock(id='toolu_01HHkNr9A7NEbC529QGBKVan', input={'order_id': 'O1'}, name='cancel_order', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 745; Out: 107; Total: 852)
- Cancelling order O1
ToolsBetaMes

Both order cancellations were successful. I have now cancelled all orders for customer C1.

<details>

- id: msg_01QwVE8zgbGm3QVjxSjyU229
- content: [{'text': 'Both order cancellations were successful. I have now cancelled all orders for customer C1.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 933, 'output_tokens': 23}

</details>

OK Claude thinks the orders were cancelled -- let's check one:

In [ ]:
chat.toolloop('What is the status of order O2?')

- Retrieving order O2


The status of order O2 is now 'Cancelled' since I successfully cancelled that order earlier.

<details>

- id: msg_01Mnq3igKL5a6mWzTa3p5hxg
- content: [{'text': "The status of order O2 is now 'Cancelled' since I successfully cancelled that order earlier.", 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 1095, 'output_tokens': 26}

</details>

## Code interpreter

In [ ]:
from lmtools import python
import traceback

In [ ]:
from IPython import InteractiveShell

TODO:

- pass code directly to run in shell
- use shell
- download notebook when done, or open in colab
- image outputs

In [ ]:
imps = 'import warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses\n'

In [ ]:
def call_python(code:str,   # Code to execute
                xtra:int=0, # Leave this as default unless requested otherwise
               ): # Result of last node, if it's an expression, or `None` otherwise
    """Executes python `code` with `timeout` and returns the contents of the expression on the last line.
    The user is prompted for whether to proceed. Raises `SystemError` if user declines.
    Raised exceptions are returned as in string form, with a stack trace.

    Example:
    
    >>> call_python('math.factorial(4)')
    24"""
    print(code)
    go = input(f'Press Enter to execute, or enter "n" to stop?\n```\n{code}\n```\n')
    try:
        if go: raise SystemError('User declined request to run code')
        if not xtra: raise RuntimeError('Subsystem init incomplete; please re-issue request with `xtra=1`')
        res = python(imps+code)
    except Exception as e: res = traceback.format_exc()
    print('Return value: ', res)
    return res

In [ ]:
tools = [call_python]
sp = '''If you write code that results in an error (except if it's due to the user declining the request to execute code), then attempt to fix it yourself and re-run Python with your new code.'''
chat = Chat(models[-1], tools=tools, sp=sp)

In [ ]:
chat.h = [
    'If I wanted you to call a function in the `math` module using your tool, would you need to import it?',
    'No, because the `call_python` tool information shows that it is already in the symbol table.',
    'Would you print the result?',
    'No, because `call_python` says I should just write the expression directly, like in Jupyter.']

In [ ]:
def _show_cts(r): print(contents(r))

In [ ]:
chat.toolloop('''What is the value of `13!`?''', trace_func=print, temp=0.2)

ToolsBetaMessage(id='msg_01J2Febihvdpmw3qxFj38YRb', content=[TextBlock(text="Okay, let's use the `call_python` tool to calculate the value of 13!:", type='text'), ToolUseBlock(id='toolu_01SZtx6yhGeWS6oRuopty3nf', input={'code': 'math.factorial(13)'}, name='call_python', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 582; Out: 80; Total: 662)
math.factorial(13)
Press Enter to execute, or enter "n" to stop?
```
math.factorial(13)
```
n
Return value:  Traceback (most recent call last):
  File "/tmp/ipykernel_819945/959138007.py", line 15, in call_python
    if go: raise SystemError('User declined request to run code')
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
SystemError: User declined request to run code

ToolsBetaMessage(id='msg_01DZPNwX5uBTe4K9qVh7JWrk', content=[TextBlock(text='Oops, looks like I need to get your permission first before running the code. Are you okay wi

Oops, looks like I need to get your permission first before running the code. Are you okay with me executing the Python code to calculate 13!?

<details>

- id: msg_01DZPNwX5uBTe4K9qVh7JWrk
- content: [{'text': 'Oops, looks like I need to get your permission first before running the code. Are you okay with me executing the Python code to calculate 13!?', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 747, 'output_tokens': 37}

</details>

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()